# Build Your First Question Answering System

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deepset-ai/haystack-tutorials/blob/main/tutorials/01_Basic_QA_Pipeline.ipynb)

Question answering allows you to quickly look into your Document collections to find answers to your questions. You can use it to search through complex knowledge bases, or long documents.

A knowledge base could, for example, be your website, an internal wiki or a collection of financial reports. In this tutorial we will work on a set of wiki pages about Game of Thrones. Let's learn how to build a question answering system and discover more about the marvellous seven kingdoms!


## Prepare Environment

Before running the code in this notebook, you should set up the Colab environment with the following steps:
- [Enable GPU Runtime in GPU]()
- [Check if GPU is Enabled]()
- [Set logging level to INFO]()


## Haystack Installation

To start, let's install the latest release of Haystack with `pip`:

In [ ]:
%%bash

pip install --upgrade pip
pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

## Document Store

A Haystack question answering system finds answers to questions within the documents stored in a `DocumentStore`. In this tutorial, we are initializing an `ElasticsearchDocumentStore` but there are many other options available. To learn which one is right for your use case, and how to initialize it, see [Choosing the Right Document Store](https://haystack.deepset.ai/components/document-store#choosing-the-right-document-store) and [Initialization](https://haystack.deepset.ai/components/document-store#initialisation).


### Start an Elasticsearch Server

The `ElasticsearchDocumentStore` needs to attach to a running Elasticsearch server. To download, extract and set the permission for the Elasticsearch installation image, run:

In [ ]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

To start the server, run:

In [ ]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

Let's wait 30 seconds to make sure the server has fully started up.

In [ ]:
import time
time.sleep(30)

If you are working in an environment where Docker is available, you can also start Elasticsearch via Docker. This can be done [manually](https://haystack.deepset.ai/components/document-store#initialisation), or using our [`launch_es()`]() utility function.

### Create the DocumentStore

When you initialize the `ElasticsearchDocumentStore`, it opens a connection with the Elasticsearch service.

In [ ]:
import os
from haystack.document_stores import ElasticsearchDocumentStore

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")

document_store = ElasticsearchDocumentStore(
    host=host,
    username="",
    password="",
    index="document"
)

## Preparing Documents

Let's download 517 articles from the Game of Thrones Wikipedia. They can be found in `data/tutorial1` as a set of `.txt` files.

In [ ]:
from haystack.utils import fetch_archive_from_http

doc_dir = "data/tutorial1"

fetch_archive_from_http(
    url="https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt1.zip",
    output_dir=doc_dir
)

The `.txt` files we just downloaded need to be converted into Haystack [Document objects](https://haystack.deepset.ai/components/documents-answers-labels#document) before they can be written into the DocumentStore. We will also apply the `clean_wiki_text` cleaning function to the text and split the Wikipedia documents by paragraph breaks.

In [ ]:
from haystack.utils import clean_wiki_text, convert_files_to_docs
docs = convert_files_to_docs(
    dir_path=doc_dir,
    clean_func=clean_wiki_text,
    split_paragraphs=True
)

Now let's write these Documents into the DocumentStore.

In [ ]:
# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(docs)

While the default code in this tutorial uses Game of Thrones data, you can also supply your own data. So long as your data adheres to the [input format](https://haystack.deepset.ai/components/document-store#input-format) or is cast into a [Document object](https://haystack.deepset.ai/components/documents-answers-labels#document), it can be written into the DocumentStore.

## Retriever

Retrievers sift through all the Documents and return only those that it thinks might contain the answer to the question. Since this happens at [query time](), they need to be fast. Here we are using the BM25 algorithm which is considered a [sparse retrieval method](https://haystack.deepset.ai/pipeline_nodes/retriever#deeper-dive-dense-vs-sparse). For more Retriever options, see [Retriever](https://haystack.deepset.ai/pipeline_nodes/retriever).

In [ ]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

## Reader

A Reader scans the texts returned by Retrievers in detail and extracts the top answer candidates. Readers are based on powerful deep learning models but are much slower than Retrievers at processing the same amount of text. Haystack Readers can load question answering models from [Hugging Face's model hub](https://huggingface.co/models?pipeline_tag=question-answering&sort=downloads). Here we are using a base sized RoBERTa question answering model called [`deepset/roberta-base-squad2`](https://huggingface.co/deepset/roberta-base-squad2). To find out what model works best for your use case, see [Models](https://haystack.deepset.ai/pipeline_nodes/reader#models).

In [ ]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

## The Retriever-Reader Pipeline

The Retriever and Reader that we just initialized are considered [nodes](https://haystack.deepset.ai/pipeline_nodes/overview) in Haystack and they are connected using a [`pipeline`](https://haystack.deepset.ai/components/pipelines). Pipelines are customizable, giving you the power to define how data is routed through the nodes at both indexing and querying time.

It makes sense to join a Retriever and Reader because they are a complementary pairing. While the Reader is very effective at picking out answers to questions, it is not fast enough to perform this on large amounts of text at query time. By performing retrieval first, only the most promising candidate Documents are passed to the Reader, thus reducing its workload. This improvement in speed can come with a small tradeoff in accuracy. To learn how to optimize the performance of your question answering system, have a look at [Optimization](https://haystack.deepset.ai/guides/optimization).

In Haystack, there is a `Pipeline` class that allows you to define your own pipeline configuration. However, there are also [Ready-Made Pipelines](https://haystack.deepset.ai/components/ready-made-pipelines) that simplify the initialization of commonly used configurations. Here, we are using the [`ExtractiveQAPipeline`](https://haystack.deepset.ai/components/ready-made-pipelines#extractiveqapipeline) that combines our Reader and Retriever.


In [ ]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

## Asking Questions and Getting Answers

Haystack pipelines have a `run()` method that performs a query, or in the case of an `ExtractiveQAPipeline`, answers a question. The `params` argument allows you to provide arguments to the nodes performing the query. See [Arguments](https://haystack.deepset.ai/components/pipelines#arguments) to learn how to populate this field, and [Choosing the Right top-k Values] to understand what the `top-k` parameters are doing.


In [ ]:
prediction = pipe.run(
    query="Who is the father of Arya Stark?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
)

Here are some questions you could try out:
- Who is the father of Arya Stark?
- Who created the Dothraki vocabulary?
- Who is the sister of Sansa?

The answers returned by the pipeline can be printed out directly.

In [ ]:
from pprint import pprint

pprint(prediction)

We also provide a utility functions to simplify the output.

In [ ]:
from haystack.utils import print_answers

print_answers(
    prediction,
    details="minimum" ## Choose from `minimum`, `medium` and `all`
)

And there you have it! Congratulations on building your first machine learning based question answering system!

## About us

This [Haystack](https://github.com/deepset-ai/haystack/) notebook was made with love by [deepset](https://deepset.ai/) in Berlin, Germany

We bring NLP to the industry via open source!  
Our focus: Industry specific language models & large scale QA systems.  
  
Some of our other work: 
- [German BERT](https://deepset.ai/german-bert)
- [GermanQuAD and GermanDPR](https://deepset.ai/germanquad)
- [FARM](https://github.com/deepset-ai/FARM)

Get in touch:
[Twitter](https://twitter.com/deepset_ai) | [LinkedIn](https://www.linkedin.com/company/deepset-ai/) | [Discord](https://haystack.deepset.ai/community/join) | [GitHub Discussions](https://github.com/deepset-ai/haystack/discussions) | [Website](https://deepset.ai)

By the way: [we're hiring!](https://www.deepset.ai/jobs)
